![](https://www.finplus.co.in/wp-content/uploads/2017/10/Top-Categories-In-Online-Retail-In-India.jpg)

## Genel Bakış
<a href="https://archive.ics.uci.edu/ml/datasets/online+retail"> Çevrimiçi perakende (Online retail), İngiltere merkezli ve kayıtlı mağaza dışı çevrimiçi perakende için 01/12/2010 ile 09/12/2011 arasında gerçekleşen tüm işlemleri içeren uluslararası bir veri setidir </a> Şirket esas olarak hediyelik eşyalar satıyor ve şirketin birçok müşterisi de toptancıdır.

## İş Hedefi
Şirketin müşterilerini verimli bir şekilde hedefleyebilmesi için müşterilerin segmentlere ayrılması amaçlanmaktadır.


## Adım 1 : Veriyi Okuma ve Anlama

In [ ]:
# import required libraries for dataframe and visualization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day7-DecisionTree/notebooks"
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART1/Day8-UnsupervisedLearning/notebooks"

DATASET_PATH = ROOT_DIR + "/datasets/"

In [ ]:
# Reading the data on which analysis needs to be done

retail = pd.read_csv(DATASET_PATH + 'OnlineRetail.csv', sep=",", encoding="ISO-8859-1", header=0)
retail.head()

In [ ]:
# shape of df

retail.shape

In [ ]:
# df info

retail.info()

In [ ]:
# df description

retail.describe()

## Adım 2 : Veri Temizleme

In [ ]:
# Calculating the Missing Values % contribution in DF

df_null = round(100*(retail.isnull().sum())/len(retail), 2)
df_null

In [ ]:
# Droping rows having missing values

retail = retail.dropna()
retail.shape

In [ ]:
# Changing the datatype of Customer Id as per Business understanding

retail['CustomerID'] = retail['CustomerID'].astype(str)

## Adım 3 : Veri Hazırlama

#### Müşterileri aşağıdaki 3 faktöre göre analiz edeceğiz:
- R (Yenilik-Recency): Son satın alma tarihinden itibaren geçen gün sayısı
- F (Sıklık-Frequency): İşlem sayısı
- M (Parasal-Monetary): Toplam işlem tutarı (gelir katkısı)


In [ ]:
# Yeni Öznitelik : Monetary
retail['Amount'] = retail['Quantity']*retail['UnitPrice']

rfm_m = retail.groupby('CustomerID')['Amount'].sum()

rfm_m = rfm_m.reset_index()
rfm_m.head()

In [ ]:
# Yeni Öznitelik : Frequency

rfm_f = retail.groupby('CustomerID')['InvoiceNo'].count()
rfm_f = rfm_f.reset_index()
rfm_f.columns = ['CustomerID', 'Frequency']
rfm_f.head()

In [ ]:
# Merging the two dfs

rfm = pd.merge(rfm_m, rfm_f, on='CustomerID', how='inner')
rfm.head()

In [ ]:
# Yeni Öznitelik : Recency

# datetime'ı uygun datatype'a dönüştürme

retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'],format='%d-%m-%Y %H:%M')

In [ ]:
# Son işlem tarihini bilmek için maksimum tarihi hesaplama

max_date = max(retail['InvoiceDate'])
max_date

In [ ]:
# Maksimum tarih ile işlem tarihi arasındaki farkı hesaplama

retail['Diff'] = max_date - retail['InvoiceDate']
retail.head()

In [ ]:
# Müşterilerin güncelliğini almak için son işlem tarihini hesaplama

rfm_p = retail.groupby('CustomerID')['Diff'].min()
rfm_p = rfm_p.reset_index()
rfm_p.head()

In [ ]:
# Yalnızca gün sayısını ayıkla

rfm_p['Diff'] = rfm_p['Diff'].dt.days
rfm_p.head()

In [ ]:
# Nihai RFM dataframe'i elde etmek için dataframe'leri birleştirelim

rfm = pd.merge(rfm, rfm_p, on='CustomerID', how='inner')
rfm.columns = ['CustomerID', 'Amount', 'Frequency', 'Recency']
rfm.head()

#### 2 tür aykırı değer vardır ve aykırı değerleri veri setimizi çarpıtabileceği için ele alacağız.
- İstatistiksel
- Alana özgü

In [ ]:
# Outlier Analysis of Amount Frequency and Recency

attributes = ['Amount','Frequency','Recency']
plt.rcParams['figure.figsize'] = [10,8]

#sns.boxplot(data = rfm[attributes], orient="v", palette="Set2", whis=1.5, saturation=1, width=0.7)
sns.boxplot(data = rfm[attributes])

plt.title("Outliers Variable Distribution", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attributes", fontweight = 'bold')

In [ ]:
# Removing (statistical) outliers for Amount
Q1 = rfm.Amount.quantile(0.05)
Q3 = rfm.Amount.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Amount >= Q1 - 1.5*IQR) & (rfm.Amount <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Recency
Q1 = rfm.Recency.quantile(0.05)
Q3 = rfm.Recency.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Recency >= Q1 - 1.5*IQR) & (rfm.Recency <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Frequency
Q1 = rfm.Frequency.quantile(0.05)
Q3 = rfm.Frequency.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Frequency >= Q1 - 1.5*IQR) & (rfm.Frequency <= Q3 + 1.5*IQR)]

### Nitelikleri Yeniden Ölçeklendirme

Değişkenleri karşılaştırılabilir bir ölçeğe sahip olacak şekilde yeniden ölçeklendirmek son derece önemlidir.|
Yeniden ölçeklendirmenin iki yaygın yolu vardır:

1. Min-Max ölçeklendirme
2. Standardizasyon (mean-0, sigma-1) 

Burada Standardization Scaling'i kullanacağız.

In [ ]:
# Rescaling the attributes

rfm_df = rfm[['Amount', 'Frequency', 'Recency']]

# Instantiate
scaler = StandardScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_df)
rfm_df_scaled.shape

In [ ]:
rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.columns = ['Amount', 'Frequency', 'Recency']
rfm_df_scaled.head()

## Step 4 : Modeli Oluşturma

### K-Means Kümeleme

K-means kümeleme, en basit ve popüler denetimsiz makine öğrenimi algoritmalarından biridir.<br>

Algoritma şu şekilde çalışır:

- İlk önce, ortalama (means) olarak adlandırılan k noktayı rastgele atıyoruz.
- Her bir öğeyi en yakın merkeze göre kategorize ediyoruz ve o merkeze göre kategorize edilen öğelerin o ana kadarki ortalamaları olan küme merkezinin koordinatlarını güncelliyoruz.
- İşlemi belirli sayıda yineleme için tekrarlıyoruz ve sonunda kümelerimizi oluşturmuş oluyoruz.

In [ ]:
# k-means with some arbitrary k

kmeans = KMeans(n_clusters=4, max_iter=50)
kmeans.fit(rfm_df_scaled)

In [ ]:
kmeans.labels_

### Optimal Küme Sayısını Bulma

#### Doğru sayıda Küme elde etmek için Dirsek Eğrisi (Elbow Curve)
Denetimsiz herhangi bir algoritma için temel bir adım, verilerin kümelenebileceği en uygun küme sayısını belirlemektir. Dirsek Yöntemi (Elbow Curve), bu optimal k değerini belirlemek için en popüler yöntemlerden biridir.

In [ ]:
# Elbow-curve/SSD

ssd = []
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for num_clusters in range_n_clusters:
  
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_df_scaled)
    
    ssd.append(kmeans.inertia_)
    
# plot the SSDs for each n_clusters
plt.plot(ssd)

### Siluet Analizi (Silhouette Analysis)

$$\text{silhouette score}=\frac{p-q}{max(p,q)}$$

$p$, veri noktasının parçası olmadığı en yakın kümedeki noktalara olan ortalama uzaklıktır.

$q$, kendi kümesindeki tüm noktalara küme içi ortalama uzaklıktır.

* Siluet puan aralığının değeri -1 ile 1 arasındadır.

* 1'e yakın bir puan, veri noktasının kümedeki diğer veri noktalarına çok benzer olduğunu gösterir,

* -1'e yakın bir puan, veri noktasının kümesindeki veri noktalarına benzemediğini gösterir.

In [ ]:
# Silhouette analysis
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:
    
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_df_scaled)
    
    cluster_labels = kmeans.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(rfm_df_scaled, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))
    
    

In [ ]:
# Final model with k=3
kmeans = KMeans(n_clusters=3, max_iter=50)
kmeans.fit(rfm_df_scaled)

In [ ]:
 kmeans.labels_

In [ ]:
# assign the label
rfm['Cluster_Id'] = kmeans.labels_
rfm.head()

In [ ]:
# Box plot to visualize Cluster Id vs Frequency

sns.boxplot(x='Cluster_Id', y='Amount', data=rfm)

In [ ]:
# Box plot to visualize Cluster Id vs Frequency

sns.boxplot(x='Cluster_Id', y='Frequency', data=rfm)

In [ ]:
# Box plot to visualize Cluster Id vs Recency

sns.boxplot(x='Cluster_Id', y='Recency', data=rfm)

### Hiyerarşik Kümeleme

Hiyerarşik kümeleme, yukarıdan aşağıya önceden belirlenmiş bir sıralamaya sahip kümeler oluşturmayı içerir. Örneğin, sabit diskteki tüm dosya ve klasörler bir hiyerarşi içinde düzenlenir. İki tür hiyerarşik kümeleme vardır,
- Bölücü
- Aglomeratif.

**Tek Bağlantı (Single Linkage):<br>**

Tek bağlantılı hiyerarşik kümelemede, iki küme arasındaki mesafe, her kümedeki iki nokta arasındaki en kısa mesafe olarak tanımlanır. Örneğin, soldaki “r” ve “s” kümeleri arasındaki mesafe, en yakın iki noktası arasındaki okun uzunluğuna eşittir.
![](https://www.saedsayad.com/images/Clustering_single.png)

In [ ]:
# Single linkage: 

mergings = linkage(rfm_df_scaled, method="single", metric='euclidean')
dendrogram(mergings)
plt.show()

**Tam Bağlantı (Complete Linkage)<br>**

Tam bağlantı hiyerarşik kümelemede, iki küme arasındaki mesafeyi her kümedeki iki nokta arasındaki en uzun mesafe olarak tanımlar. Örneğin, soldaki “r” ve “s” kümeleri arasındaki mesafe, en uzak iki noktası arasındaki okun uzunluğuna eşittir.
![](https://www.saedsayad.com/images/Clustering_complete.png)

In [ ]:
# Complete linkage

mergings = linkage(rfm_df_scaled, method="complete", metric='euclidean')
dendrogram(mergings)
plt.show()

**Ortalama bağlantı (Average Linkage):<br>**

Ortalama bağlantı hiyerarşik kümelemede, iki küme arasındaki mesafeyi bir kümedeki her nokta ile diğer kümedeki her nokta arasındaki ortalama mesafe olarak tanımlar. Örneğin, soldaki “r” ve “s” kümeleri arasındaki mesafe, bir kümenin noktalarını diğerine bağlayan her bir okun ortalama uzunluğuna eşittir.

![](https://www.saedsayad.com/images/Clustering_average.png)

In [ ]:
# Average linkage

mergings = linkage(rfm_df_scaled, method="average", metric='euclidean')
dendrogram(mergings)
plt.show()

#### Dendrogramın K'ye göre kesilmesi

In [ ]:
# 3 clusters
cluster_labels = cut_tree(mergings, n_clusters=3).reshape(-1, )
cluster_labels

In [ ]:
# Assign cluster labels

rfm['Cluster_Labels'] = cluster_labels
rfm.head()

In [ ]:
# Plot Cluster Id vs Amount

sns.boxplot(x='Cluster_Labels', y='Amount', data=rfm)

In [ ]:
# Plot Cluster Id vs Frequency

sns.boxplot(x='Cluster_Labels', y='Frequency', data=rfm)

In [ ]:
# Plot Cluster Id vs Recency

sns.boxplot(x='Cluster_Labels', y='Recency', data=rfm)

## Adım 5 : Final Analizi

### Çıkarım:
3 Küme Kimliği (Ids) ile K-Means Kümeleme:

- Küme Kimliği 1 olan müşteriler, diğer müşterilere göre yüksek miktarda işlem yapan müşterilerdir.
- Küme Kimliği 1 olan müşteriler sık alıcılardır.
- Küme Kimliği 2 olan müşteriler yeni alıcılar değildir ve bu nedenle iş açısından en az öneme sahiptir.

3 Küme Etiketli Hiyerarşik Kümeleme:

- Cluster_Labels 2'ye sahip müşteriler, diğer müşterilere kıyasla yüksek miktarda işlem yapan müşterilerdir.
- Cluster_Labels 2'ye sahip müşteriler sık alıcılardır.
- Cluster_Labels 0'a sahip müşteriler yeni alıcılar değildir ve bu nedenle iş açısından en az öneme sahiptir.